### Compare Listing 
<ol>
<li>a: vector uniform</li>
<li>b: greedy</li>
<li>c: e - greedy</li>
<li>d: decay e - greedy</li>
<li>e: Linear Reward Inaction (Pursuit Methods)</li>
<li>f: Linear Reward Penalty (Pursuit Methods)</li>
<li>g: UBC 1</li>
<li>h: UCB 1-Tuned</li>
<li>i: Thompson Sampling (beta)</li>
<li>j: Thompson Sampling (uniform)</li>
<li>k: Neural Network</li>
<li>l: softmax </li>
<li>m: Gradient Bandits</li>
<li>n: Non Stationary</li>
</ol>

In [2]:
# import lib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy,time,sys
import scipy.stats as stats
from scipy.stats import beta
np.random.seed(5678)
np.set_printoptions(3)
tf.set_random_seed(678)
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [3]:
# setting the ground truth
num_bandit = 12
num_ep  = 20
num_iter= 1000
gt_prob = np.random.uniform(0,1,num_bandit)
optimal_choice = np.argmax(gt_prob)
print(gt_prob)
print('Best Choice: ',optimal_choice,gt_prob[optimal_choice])

[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Best Choice:  11 0.7364685816073836


In [19]:
# a vectorized
a_expect = np.zeros((num_ep,num_bandit))
                    
for eps in range(num_ep):
    temp_expect = np.zeros(num_bandit)
    temp_choice = np.zeros(num_bandit)
                    
    for iter in range(num_iter//10):
        temp_choice    = temp_choice + 1
        current_reward = np.random.uniform(0,1,num_bandit) < gt_prob
        temp_expect    = temp_expect + current_reward

    a_expect[eps,:] = temp_expect/temp_choice
                    
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(a_expect.mean(0))

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[0.481 0.057 0.354 0.528 0.599 0.423 0.17  0.275 0.084 0.182 0.084 0.758]


In [62]:
# b greedy
b_pull_count   = np.zeros((num_ep,num_bandit))
b_estimation   = np.zeros((num_ep,num_bandit))
b_reward       = np.zeros((num_ep,num_iter))
b_optimal_pull = np.zeros((num_ep,num_iter))
b_regret_total = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
    temp_regret = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(temp_estimation)
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + (1/(temp_pull_count[current_choice]+1)) * (current_reward-temp_estimation[current_choice])
        
        # update reward and optimal choice
        temp_reward[iter] = current_reward if iter == 0 else temp_reward[iter-1] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        temp_regret[iter] = gt_prob[optimal_choice] - gt_prob[current_choice] if iter == 0 else temp_regret[iter-1] + (gt_prob[optimal_choice] - gt_prob[current_choice])
        
    b_pull_count[eps,:]   = temp_pull_count
    b_estimation[eps,:]   = temp_estimation
    b_reward[eps,:]       = temp_reward
    b_optimal_pull[eps,:] = temp_optimal_pull
    b_regret_total[eps,:] = temp_regret
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(b_estimation.mean(0))

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[0.49 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]


In [63]:
# c e greedy 
c_pull_count   = np.zeros((num_ep,num_bandit))
c_estimation   = np.zeros((num_ep,num_bandit))
c_reward       = np.zeros((num_ep,num_iter))
c_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    epsilon = np.random.uniform(0,1)
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(temp_expect) if epsilon < np.random.uniform(0,1) else np.random.choice(np.arange(num_bandit))
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + (1/(temp_pull_count[current_choice]+1)) * (current_reward-temp_estimation[current_choice])
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
    c_pull_count[eps,:]   = temp_pull_count
    c_estimation[eps,:]   = temp_estimation
    c_reward[eps,:]       = temp_reward
    c_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(c_estimation.mean(0))

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[0.471 0.078 0.385 0.489 0.581 0.401 0.172 0.255 0.092 0.162 0.095 0.743]


In [64]:
# d decy e greedy 
d_pull_count   = np.zeros((num_ep,num_bandit))
d_estimation   = np.zeros((num_ep,num_bandit))
d_reward       = np.zeros((num_ep,num_iter))
d_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    epsilon = 1.0
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(temp_expect) if epsilon < np.random.uniform(0,1) else np.random.choice(np.arange(num_bandit))
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + (1/(temp_pull_count[current_choice]+1)) * (current_reward-temp_estimation[current_choice])
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
        # decay the eps
        epsilon = 0.999 * epsilon
        
    d_pull_count[eps,:]   = temp_pull_count
    d_estimation[eps,:]   = temp_estimation
    d_reward[eps,:]       = temp_reward
    d_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(d_estimation.mean(0))

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[0.471 0.066 0.344 0.512 0.575 0.43  0.183 0.279 0.066 0.179 0.101 0.732]


In [65]:
# e Linear Reward Inaction
e_pull_count   = np.zeros((num_ep,num_bandit))
e_estimation   = np.zeros((num_ep,num_bandit))
e_reward       = np.zeros((num_ep,num_iter))
e_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    learning_rate = 0.001
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit) + 1.0/num_bandit
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
    
    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.random.choice(num_bandit, p=temp_estimation)
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        
        mask = np.zeros(num_bandit)
        mask[current_choice] = 1.0
        
        if current_reward == 1.0:
            temp_estimation = (mask) * (temp_estimation + learning_rate * (1-temp_estimation)) + (1-mask) * ( (1-learning_rate) * temp_estimation)
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
    e_pull_count[eps,:]   = temp_pull_count
    e_estimation[eps,:]   = temp_estimation
    e_reward[eps,:]       = temp_reward
    e_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(e_estimation.mean(0))
print('Expected Normalized')
print(e_estimation.mean(0) * gt_prob.sum())

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[0.095 0.062 0.086 0.098 0.104 0.09  0.069 0.078 0.063 0.07  0.063 0.123]
Expected Normalized
[0.379 0.247 0.343 0.391 0.416 0.359 0.278 0.312 0.251 0.282 0.254 0.494]


In [66]:
# f Linear Reward Penalty
f_pull_count   = np.zeros((num_ep,num_bandit))
f_estimation   = np.zeros((num_ep,num_bandit))
f_reward       = np.zeros((num_ep,num_iter))
f_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    alpha = 0.001
    beta  = 0.0001
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit) + 1.0/num_bandit
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
    
    for iter in range(num_iter):

        # select bandit / get reward /increase count / update estimate
        current_choice = np.random.choice(num_bandit, p=temp_estimation)
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1

        mask = np.zeros(num_bandit)
        mask[current_choice] = 1.0
        
        if current_reward == 1.0:
            temp_estimation = (mask) * (temp_estimation + alpha * (1-temp_estimation)) + (1-mask) * ( (1-alpha) * temp_estimation)
        else: 
            temp_estimation = (mask) * ((1-beta) * temp_estimation) + (1-mask) * ( beta/(num_bandit-1) + (1-beta) * temp_estimation )

        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
    f_pull_count[eps,:]   = temp_pull_count
    f_estimation[eps,:]   = temp_estimation
    f_reward[eps,:]       = temp_reward
    f_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(f_estimation.mean(0))
print('Expected Normalized')
print(f_estimation.mean(0) * gt_prob.sum())

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[0.093 0.063 0.085 0.099 0.105 0.088 0.071 0.078 0.063 0.07  0.065 0.12 ]
Expected Normalized
[0.374 0.251 0.34  0.396 0.423 0.353 0.283 0.313 0.254 0.28  0.259 0.483]


In [67]:
# g UBC
g_pull_count   = np.zeros((num_ep,num_bandit))
g_estimation   = np.zeros((num_ep,num_bandit))
g_reward       = np.zeros((num_ep,num_iter))
g_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(temp_estimation + np.sqrt(2*np.log(iter+1)/(1+temp_pull_count)))
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + (1/(temp_pull_count[current_choice]+1)) * (current_reward-temp_estimation[current_choice])
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
    g_pull_count[eps,:]   = temp_pull_count
    g_estimation[eps,:]   = temp_estimation
    g_reward[eps,:]       = temp_reward
    g_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(g_estimation.mean(0))

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[0.484 0.056 0.338 0.505 0.569 0.402 0.159 0.24  0.053 0.174 0.071 0.734]


In [79]:
# h UBC Tuned
h_pull_count   = np.zeros((num_ep,num_bandit))
h_estimation   = np.zeros((num_ep,num_bandit))
h_reward       = np.zeros((num_ep,num_iter))
h_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    temp_pull_count   = np.zeros(num_bandit) 
    temp_estimation   = np.zeros(num_bandit) 
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
#         current_min_value = np.amin(np.ones(num_bandit)*0.25,temp_estimation^2 + np.sqrt((2*np.log(iter))/temp_pull_count))
        current_min_value = 0.25
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(temp_estimation + np.sqrt(np.log(iter+1)/(temp_pull_count+1e-8))*current_min_value)
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + current_reward
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
    h_pull_count[eps,:]   = temp_temp_pull_cou
    h_estimation[eps,:]   = temp_estimation + np.sqrt(np.log(iter+1)/(temp_pull_count))*current_min_value
    h_reward[eps,:]       = temp_reward
    h_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(h_estimation.mean(0))

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[143.399   0.657  54.002 181.984  31.502   1.097   0.743   0.907   0.757
   0.938   0.707 111.762]


In [69]:
# i Thompson Sampling (beta) (slow)
k_pull_count   = np.zeros((num_ep,num_bandit))
k_estimation   = np.zeros((num_ep,num_bandit))
k_reward       = np.zeros((num_ep,num_iter))
k_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):

    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
        theta_samples = [stats.beta(a=1+w,b=1+t-w).rvs(size=1) for t, w in zip(temp_pull_count, temp_estimation)]
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(theta_samples)
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + current_reward
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
    k_pull_count[eps,:]   = temp_pull_count
    k_estimation[eps,:]   = theta_samples
    k_reward[eps,:]       = temp_reward
    k_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(k_estimation.mean(0))

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[0.412 0.141 0.367 0.409 0.588 0.402 0.194 0.325 0.194 0.233 0.214 0.743]


In [70]:
# j Thompson Sampling (uniform) (slow)
k_pull_count   = np.zeros((num_ep,num_bandit))
k_estimation   = np.zeros((num_ep,num_bandit))
k_reward       = np.zeros((num_ep,num_iter))
k_optimal_pull = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):

    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
                    
    for iter in range(num_iter):
        
        theta_samples = [stats.uniform(w/(t+0.000000001),1-w/(t+0.000000001)).rvs(size=1) for t, w in zip(temp_pull_count, temp_estimation)]
        
        # select bandit / get reward /increase count / update estimate
        current_choice = np.argmax(theta_samples)
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + current_reward
        
        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
    k_pull_count[eps,:]   = temp_pull_count
    k_estimation[eps,:]   = theta_samples
    k_reward[eps,:]       = temp_reward
    k_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(k_estimation.mean(0))

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[0.702 0.469 0.715 0.747 0.768 0.701 0.648 0.616 0.503 0.516 0.603 0.885]


In [71]:
# k neural network (with adam)
k_pull_count   = np.zeros((num_ep,num_bandit))
k_estimation   = np.zeros((num_ep,num_bandit))
k_reward       = np.zeros((num_ep,num_iter))
k_optimal_pull = np.zeros((num_ep,num_iter))
            
def sigmoid(x): return 1/(1+np.exp(-x))
def d_sigmoid(x): return sigmoid(x)*(1-sigmoid(x))

for eps in range(num_ep):
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
    
    weights = np.random.randn(num_bandit,1)
    moment  = np.zeros_like(weights); 
    velocity = np.zeros_like(weights);
    epsilon  = 1.0 

    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        if np.random.uniform(0,1)>epsilon:
            current_choice = np.argmax(weights)
            current_input  = np.zeros((1,num_bandit))
            current_input[0,current_choice] = 1
        else:
            current_choice = np.random.choice(np.arange(num_bandit))
            current_input  = np.zeros((1,num_bandit))
            current_input[0,current_choice] = 1

        layer1 = current_input @ weights
        layer1a= sigmoid(layer1)

        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_estimation[current_choice] = temp_estimation[current_choice] + current_reward
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        
        # KL Divergence https://timvieira.github.io/blog/post/2014/10/06/kl-divergence-as-an-objective-function/
        grad3 = np.log(layer1a+0.0000001) - np.log(temp_estimation[current_choice]/(temp_pull_count[current_choice])+0.0000001)
        grad2 = d_sigmoid(layer1)
        grad1 = current_input
        grad  = grad1.T @ (grad3 * grad2)
        
        moment   = 0.9*moment + (1-0.9) * grad
        velocity = 0.999*velocity + (1-0.999) * grad**2
        moment_hat   = moment/(1-0.9)
        velocity_hat = velocity/(1-0.999)
        weights  = weights - 0.08 * (moment_hat/(np.sqrt(velocity_hat)+1e-8))

        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
        # Decay the learning rate
        epsilon = epsilon * 0.999
        
    k_pull_count[eps,:]   = temp_pull_count
    k_estimation[eps,:]   = np.squeeze(sigmoid(weights))
    k_reward[eps,:]       = temp_reward
    k_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(k_estimation.mean(0))
print('Scaled ')
print((gt_prob.max()-gt_prob.min())*(k_estimation.mean(0)-k_estimation.mean(0).min())/(k_estimation.mean(0).max()-k_estimation.mean(0).min()) + gt_prob.min())

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[0.468 0.073 0.311 0.505 0.543 0.373 0.13  0.27  0.071 0.164 0.08  0.7  ]
Scaled 
[0.487 0.061 0.318 0.527 0.568 0.384 0.123 0.273 0.059 0.159 0.069 0.736]


In [72]:
# l softmax
l_pull_count   = np.zeros((num_ep,num_bandit))
l_estimation   = np.zeros((num_ep,num_bandit))
l_reward       = np.zeros((num_ep,num_iter))
l_optimal_pull = np.zeros((num_ep,num_iter))
l_regret_total = np.zeros((num_ep,num_iter))
                    
for eps in range(num_ep):
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
    temp_regret = np.zeros(num_iter)
    tempture = 300
                    
    for iter in range(num_iter):
        
        # select bandit / get reward /increase count / update estimate
        pi  = np.exp(temp_estimation/tempture) / np.sum(np.exp(temp_estimation/tempture))
        cdf = np.cumsum(pi)
        current_choice = np.where(np.random.uniform(0,1) < cdf)[0][0]
        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        temp_estimation[current_choice] = temp_estimation[current_choice] + (1/(temp_pull_count[current_choice]+1)) * (current_reward-temp_estimation[current_choice])

        # update reward and optimal choice
        temp_reward[iter] = current_reward if iter == 0 else temp_reward[iter-1] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        temp_regret[iter] = gt_prob[optimal_choice] - gt_prob[current_choice] if iter == 0 else temp_regret[iter-1] + (gt_prob[optimal_choice] - gt_prob[current_choice])
        
        tempture = tempture * 0.999999
        
    l_pull_count[eps,:]   = temp_pull_count
    l_estimation[eps,:]   = temp_estimation
    l_reward[eps,:]       = temp_reward
    l_optimal_pull[eps,:] = temp_optimal_pull
    l_regret_total[eps,:] = temp_regret
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(l_estimation.mean(0))

Ground Truth
[0.489 0.059 0.366 0.519 0.598 0.431 0.179 0.285 0.071 0.185 0.088 0.736]
Expected 
[0.498 0.062 0.367 0.515 0.609 0.416 0.173 0.272 0.077 0.189 0.088 0.738]


In [39]:
# z deep neural network (with adam)
z_pull_count   = np.zeros((num_ep,num_bandit))
z_estimation   = np.zeros((num_ep,num_bandit))
z_reward       = np.zeros((num_ep,num_iter))
z_optimal_pull = np.zeros((num_ep,num_iter))
            
def sigmoid(x): return 1/(1+np.exp(-x))
def d_sigmoid(x): return sigmoid(x)*(1-sigmoid(x))
def ReLU(x): return x * (x > 0)
def dReLU(x):return 1. * (x > 0)

for eps in range(num_ep):
    temp_pull_count   = np.zeros(num_bandit)
    temp_estimation   = np.zeros(num_bandit)
    temp_reward       = np.zeros(num_iter)
    temp_optimal_pull = np.zeros(num_iter)
    
    weights1 = np.random.randn(num_bandit,num_bandit); moment1  = np.zeros_like(weights1);  velocity1 = np.zeros_like(weights1);
    weights2 = np.random.randn(num_bandit,num_bandit); moment2  = np.zeros_like(weights2);  velocity2 = np.zeros_like(weights2);
    weights3 = np.random.randn(num_bandit,num_bandit); moment3  = np.zeros_like(weights3);  velocity3 = np.zeros_like(weights3);
    epsilon  = 0
    for iter in range(num_iter):
        
        current_input = np.ones((1,num_bandit))
        layer1 = current_input @ weights1
        layer1a= ReLU(layer1)
        
        layer2 = layer1a @ weights2
        layer2a= ReLU(layer2)
        
        layer3 = layer2a @ weights3
        layer3a= sigmoid(layer3)
        
        # select bandit / get reward /increase count / update estimate
        if np.random.uniform(0,1)>epsilon:
            current_choice = np.argmax(layer3a)
            current_input  = np.zeros((1,num_bandit))
            current_input[0,current_choice] = 1
        else:
            current_choice = np.random.choice(np.arange(num_bandit))
            current_input  = np.zeros((1,num_bandit))
            current_input[0,current_choice] = 1

        current_reward = 1 if np.random.uniform(0,1) < gt_prob[current_choice] else 0
        temp_estimation[current_choice] = temp_estimation[current_choice] + current_reward
        temp_pull_count[current_choice] = temp_pull_count[current_choice] + 1
        
        grad33 = np.log(layer3a+0.0000001) - (np.log(temp_estimation[current_choice]/(temp_pull_count[current_choice])+0.0000001)*current_input)
        grad32 = d_sigmoid(layer3)
        grad31 = layer2a
        grad3  = grad31.T @ (grad33 * grad32)
        
        grad23 = (grad33 * grad32).T @ weights3
        grad22 = 
        print(grad3.shape)
        sys.exit()
        
        moment   = 0.9*moment + (1-0.9) * grad
        velocity = 0.999*velocity + (1-0.999) * grad**2
        moment_hat   = moment/(1-0.9)
        velocity_hat = velocity/(1-0.999)
        weights  = weights - 0.08 * (moment_hat/(np.sqrt(velocity_hat)+1e-8))

        # update reward and optimal choice
        temp_reward[iter] = temp_reward[iter] + current_reward
        temp_optimal_pull[iter] = 1 if current_choice == optimal_choice else 0
        
        # Decay the learning rate
        epsilon = epsilon * 0.999
        
    z_pull_count[eps,:]   = temp_pull_count
    z_estimation[eps,:]   = np.squeeze(sigmoid(weights))
    z_reward[eps,:]       = temp_reward
    z_optimal_pull[eps,:] = temp_optimal_pull
        
print('Ground Truth')
print(gt_prob)
print('Expected ')
print(z_estimation.mean(0))


(12, 12)


SystemExit: 

In [73]:
%%notify 
print('done')

done


<IPython.core.display.Javascript object>

# Reference 
1. numpy.set_printoptions — NumPy v1.14 Manual. (2019). Docs.scipy.org. Retrieved 13 January 2019, from https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.set_printoptions.html
2. [ Archived Post ] Random Note about Multi-Arm Bandit Problem 2. (2019). Medium. Retrieved 13 January 2019, from https://medium.com/@SeoJaeDuk/archived-post-random-note-about-multi-arm-bandit-problem-2-5c522d1dfbdc
3. Vieira, T. (2014). KL-divergence as an objective function — Graduate Descent. Timvieira.github.io. Retrieved 13 January 2019, from https://timvieira.github.io/blog/post/2014/10/06/kl-divergence-as-an-objective-function/
4. Some Reinforcement Learning: The Greedy and Explore-Exploit Algorithms for the Multi-Armed Bandit Framework in Python. (2019). Datasciencecentral.com. Retrieved 13 January 2019, from https://www.datasciencecentral.com/profiles/blogs/some-reinforcement-learning-the-greedy-and-explore-exploit
5. (2019). Cs.mcgill.ca. Retrieved 13 January 2019, from https://www.cs.mcgill.ca/~vkules/bandits.pdf
